In [1]:
from bert import bert_ATE
from dataset import dataset_ATM



In [2]:
from torch.utils.data import DataLoader, ConcatDataset
from transformers import BertTokenizer
import torch
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import time
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [3]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pretrain_model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(pretrain_model_name)
lr = 2e-5
model_ATE = bert_ATE(pretrain_model_name).to(DEVICE)
optimizer_ATE = torch.optim.Adam(model_ATE.parameters(), lr=lr)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
def evl_time(t):
    min, sec= divmod(t, 60)
    hr, min = divmod(min, 60)
    return int(hr), int(min), int(sec)

def load_model(model, path):
    model.load_state_dict(torch.load(path), strict=False)
    return model
    
def save_model(model, name):
    torch.save(model.state_dict(), name)

In [48]:
# df = pd.read_csv("D:/FYP_A+/Project_Update/NasiLemakReviewTags.csv")


In [5]:
# Create datasets using the split data
review_train_ds = dataset_ATM(pd.read_csv("D:/FYP_A+/Project_Update/train_dataset.csv"), tokenizer)
review_test_ds = dataset_ATM(pd.read_csv("D:/FYP_A+/Project_Update/test_dataset.csv"), tokenizer)

In [21]:
print(type(review_train_ds))

<class 'dataset.dataset_ATM'>


In [63]:
print(review_train_ds[12])


(['serving', 'only', 'nas', '##i', 'le', '##ma', '##k', ',', 'this', 'huge', 'restaurant', 'offers', 'various', 'sides', ',', 'fruit', 'juice', '##s', '/', 'smooth', '##ies', ',', 'and', 'dessert', '.', 'there', 'are', '2', 'lines', 'for', 'either', 'dining', 'in', 'or', 'take', 'out', '.', 'portions', 'were', 'not', 'too', 'big', 'and', 'the', 'food', 'was', 'delicious', '.', 'lots', 'of', 'workers', 'and', 'plenty', 'of'], tensor([ 3529,  2069, 17235,  2072,  3393,  2863,  2243,  1010,  2023,  4121,
         4825,  4107,  2536,  3903,  1010,  5909, 10869,  2015,  1013,  5744,
         3111,  1010,  1998, 18064,  1012,  2045,  2024,  1016,  3210,  2005,
         2593,  7759,  1999,  2030,  2202,  2041,  1012,  8810,  2020,  2025,
         2205,  2502,  1998,  1996,  2833,  2001, 12090,  1012,  7167,  1997,
         3667,  1998,  7564,  1997]), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0

In [6]:
def check_token_lengths(dataset):
    error_indices = []

    for idx in range(len(dataset)):
        try:
            bert_tokens, ids_tensor, tags_tensor = dataset[idx]

            is_length_match = len(bert_tokens) == len(ids_tensor) == len(tags_tensor)

            print(f"Sample {idx + 1} - Lengths Match: {is_length_match}")
        except IndexError:
            print(f"IndexError at index {idx}")
            error_indices.append(idx)

    return error_indices

# Check for the training dataset
error_indices = check_token_lengths(review_train_ds)

print("Indices causing IndexError:", error_indices)



# Check for the test dataset
# check_token_lengths(review_test_ds)



Sample 1 - Lengths Match: True
Sample 2 - Lengths Match: True
Sample 3 - Lengths Match: True
Sample 4 - Lengths Match: True
Sample 5 - Lengths Match: True
Sample 6 - Lengths Match: True
Sample 7 - Lengths Match: True
Sample 8 - Lengths Match: True
Sample 9 - Lengths Match: True
Sample 10 - Lengths Match: True
Sample 11 - Lengths Match: True
Sample 12 - Lengths Match: True
Sample 13 - Lengths Match: True
Sample 14 - Lengths Match: True
Sample 15 - Lengths Match: True
Sample 16 - Lengths Match: True
Sample 17 - Lengths Match: True
Sample 18 - Lengths Match: True
Sample 19 - Lengths Match: True
Sample 20 - Lengths Match: True
Sample 21 - Lengths Match: True
Sample 22 - Lengths Match: True
Sample 23 - Lengths Match: True
Sample 24 - Lengths Match: True
Sample 25 - Lengths Match: True
Sample 26 - Lengths Match: True
Sample 27 - Lengths Match: True
Sample 28 - Lengths Match: True
Sample 29 - Lengths Match: True
Sample 30 - Lengths Match: True
Sample 31 - Lengths Match: True
Sample 32 - Lengt

In [247]:
# Confirm the existence of indices before removal
for idx in error_indices:
    if idx < len(review_train_ds):
        print(f"Index {idx} exists in the dataset.")
    else:
        print(f"Index {idx} is out of range for the dataset.")


In [ ]:
# Remove problematic indices
indices_to_remove = error_indices.copy()  # Ensure a copy to avoid modifying the original list

# Create a new instance of dataset_ATM without the problematic indices
cleaned_review_train_ds = dataset_ATM(
    [review_train_ds[i] for i in range(len(review_train_ds)) if i not in indices_to_remove],
    tokenizer
)

# Check for the cleaned training dataset
check_token_lengths(cleaned_review_train_ds)


In [ ]:
# Remove problematic indices
indices_to_remove = error_indices

# Create a new instance of dataset_ATM without the problematic indices
cleaned_review_train_ds = dataset_ATM(
    [review_train_ds[i] for i in range(len(review_train_ds)) if i not in indices_to_remove],
    tokenizer
)

# Check for the cleaned training dataset
check_token_lengths(cleaned_review_train_ds)




In [168]:
# Print the content of the first 5 samples
for i in range(5):
    sample = review_train_ds[i]
    tokens, ids_tensor, tags_tensor = sample

    print(f"Sample {i + 1}:")
    print("Tokens:", tokens)
    print("Token IDs:", ids_tensor)
    print("Tags:", tags_tensor)
    print("\n")

Sample 1:
Tokens: ['when', 'it', 'arrives', 'around', 'before', 'dusk', ',', 'the', 'line', 'is', 'not', 'long', ',', 'but', 'here', 'is', 'a', 'sur', '##au', ',', 'do', 'not', 'worry', '.']
Token IDs: tensor([ 2043,  2009,  8480,  2105,  2077, 18406,  1010,  1996,  2240,  2003,
         2025,  2146,  1010,  2021,  2182,  2003,  1037,  7505,  4887,  1010,
         2079,  2025,  4737,  1012])
Tags: tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0])


Sample 2:
Tokens: ['when', 'it', 'arrives', 'around', 'before', 'dusk', ',', 'the', 'line', 'is', 'not', 'long', ',', 'but', 'here', 'is', 'a', 'sur', '##au', ',', 'do', 'not', 'worry', '.']
Token IDs: tensor([ 2043,  2009,  8480,  2105,  2077, 18406,  1010,  1996,  2240,  2003,
         2025,  2146,  1010,  2021,  2182,  2003,  1037,  7505,  4887,  1010,
         2079,  2025,  4737,  1012])
Tags: tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0])


Sample 3:
Tokens: ['stream', '##

In [248]:
print(review_train_ds[235])

(['the', 'place', 'was', 'okay', 'but', 'the', 'food', 'were', 'under', '##w', '##helm', '##ing', '.', 'as', 'a', 'tourist', 'it', 'did', 'not', 'appeal', 'to', 'me', 'and', 'did', 'not', 'showcase', 'the', 'cuisine', 'of', 'malaysia', '.', 'we', 'ate', 'here', 'because', 'our', 'guide', 'told', 'us', 'we', 'could', 'find', 'nas', '##i', 'le', '##ma', '##k', 'and', 'me', '##e', 'gore', '##ng', 'here', 'but', 'as', 'stated', 'above', ',', 'the', 'nas', '##i', 'le', '##ma', '##k', 'was', 'under', '##w', '##helm', '##ing', 'and', 'there', 'was', 'no', 'me', '##e', 'gore', '##ng', 'in', 'sight'], tensor([ 1996,  2173,  2001,  3100,  2021,  1996,  2833,  2020,  2104,  2860,
        24546,  2075,  1012,  2004,  1037,  7538,  2009,  2106,  2025,  5574,
         2000,  2033,  1998,  2106,  2025, 13398,  1996, 12846,  1997,  6027,
         1012,  2057,  8823,  2182,  2138,  2256,  5009,  2409,  2149,  2057,
         2071,  2424, 17235,  2072,  3393,  2863,  2243,  1998,  2033,  2063,
        13

In [249]:
def create_mini_batch(samples):
    ids_tensors = [s[1] for s in samples]
    ids_tensors = pad_sequence(ids_tensors, batch_first=True)

    tags_tensors = [s[2] for s in samples]
    tags_tensors = pad_sequence(tags_tensors, batch_first=True)

    masks_tensors = torch.zeros(ids_tensors.shape, dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(ids_tensors != 0, 1)
    
    return ids_tensors, tags_tensors, masks_tensors


In [250]:
# Create DataLoader for training
train_loader = DataLoader(review_train_ds, batch_size=5, collate_fn=create_mini_batch, shuffle=True)

# Create DataLoader for testing
test_loader = DataLoader(review_test_ds, batch_size=50, collate_fn=create_mini_batch, shuffle=False)

In [251]:
def train_model_ATE(loader, epochs):
    all_data = len(loader)
    for epoch in range(epochs):
        finish_data = 0
        losses = []
        current_times = []

        for data in loader:
            try:
                t0 = time.time()
                ids_tensors, tags_tensors, masks_tensors = data
                ids_tensors = ids_tensors.to(DEVICE)
                tags_tensors = tags_tensors.to(DEVICE)
                masks_tensors = masks_tensors.to(DEVICE)

                loss = model_ATE(ids_tensors=ids_tensors, tags_tensors=tags_tensors, masks_tensors=masks_tensors)
                losses.append(loss.item())
                loss.backward()
                optimizer_ATE.step()
                optimizer_ATE.zero_grad()

                finish_data += 1
                current_times.append(round(time.time() - t0, 3))
                current = np.mean(current_times)
                hr, min, sec = evl_time(current * (all_data - finish_data) + current * all_data * (epochs - epoch - 1))
                print('epoch:', epoch, " batch:", finish_data, "/", all_data, " loss:", np.mean(losses),
                      " hr:", hr, " min:", min, " sec:", sec)
            except IndexError:
                print("IndexError during training. Skipping this sample.")

        save_model(model_ATE, 'bert_ATE.pkl')


def test_model_ATE(loader):
    pred = []
    truth = []
    with torch.no_grad():
        for data in loader:
            try:
                ids_tensors, tags_tensors, masks_tensors = data
                ids_tensors = ids_tensors.to(DEVICE)
                tags_tensors = tags_tensors.to(DEVICE)
                masks_tensors = masks_tensors.to(DEVICE)

                outputs = model_ATE(ids_tensors=ids_tensors, tags_tensors=None, masks_tensors=masks_tensors)

                _, predictions = torch.max(outputs, dim=2)

                pred += list([int(j) for i in predictions for j in i])
                truth += list([int(j) for i in tags_tensors for j in i])
            except IndexError:
                print("IndexError during testing. Skipping this sample.")

    return truth, pred



In [252]:
# Check the number of rows in the training and testing sets
print("Number of rows in the training set:", train_df.shape[0])
print("Number of rows in the testing set:", test_df.shape[0])

Number of rows in the training set: 3934
Number of rows in the testing set: 984


In [253]:
%time train_model_ATE(train_loader, 3)

epoch: 0  batch: 1 / 787  loss: 1.2178300619125366  hr: 1  min: 54  sec: 48
epoch: 0  batch: 2 / 787  loss: 1.0419926643371582  hr: 2  min: 8  sec: 58
epoch: 0  batch: 3 / 787  loss: 0.9302248358726501  hr: 2  min: 2  sec: 1
epoch: 0  batch: 4 / 787  loss: 0.8415917158126831  hr: 1  min: 54  sec: 41
epoch: 0  batch: 5 / 787  loss: 0.7626883625984192  hr: 1  min: 53  sec: 49
epoch: 0  batch: 6 / 787  loss: 0.7305687467257181  hr: 1  min: 50  sec: 10
epoch: 0  batch: 7 / 787  loss: 0.6660657269614083  hr: 1  min: 51  sec: 54
epoch: 0  batch: 8 / 787  loss: 0.639711294323206  hr: 1  min: 51  sec: 0
epoch: 0  batch: 9 / 787  loss: 0.5985711713631948  hr: 1  min: 48  sec: 40
epoch: 0  batch: 10 / 787  loss: 0.5865313827991485  hr: 1  min: 47  sec: 45
epoch: 0  batch: 11 / 787  loss: 0.5717776173895056  hr: 1  min: 47  sec: 41
epoch: 0  batch: 12 / 787  loss: 0.5825061872601509  hr: 1  min: 45  sec: 33
epoch: 0  batch: 13 / 787  loss: 0.570311365219263  hr: 1  min: 43  sec: 54
epoch: 0  batc

In [5]:
model_ATE = load_model(model_ATE, 'bert_ATE.pkl')

In [256]:
%time x, y = test_model_ATE(test_loader)
print(classification_report(x, y, target_names=[str(i) for i in range(3)]))

Wall time: 2min 4s
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     78615
           1       0.81      0.81      0.81      4146
           2       0.81      0.76      0.78      2083

    accuracy                           0.97     84844
   macro avg       0.87      0.85      0.86     84844
weighted avg       0.97      0.97      0.97     84844



In [265]:
pip install transformers datasets

In [ ]:
pip list

In [8]:
import torch
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline

# Load Aspect Term Extraction (ATE) model
def predict_model_ATE(sentence, tokenizer):
    # ... (your existing ATE prediction code)
    # Replace this with your implementation for ATE prediction
    word_pieces = []
    tokens = tokenizer.tokenize(sentence)
    word_pieces += tokens

    ids = tokenizer.convert_tokens_to_ids(word_pieces)
    input_tensor = torch.tensor([ids]).to(DEVICE)

    with torch.no_grad():
        outputs = model_ATE(input_tensor, None, None)
        _, predictions = torch.max(outputs, dim=2)
    predictions = predictions[0].tolist()
    
    return word_pieces, predictions, outputs

# Load emotion detection model
classifier = pipeline("text-classification",model='bhadresh-savani/bert-base-uncased-emotion', return_all_scores=True)

# Function to combine ATE and emotion prediction
def ATE_emotion_prediction(text, model_ATE, tokenizer, emotion_classifier):
    terms = []
    word = ""
    
    x, y, _ = predict_model_ATE(text, tokenizer)
    
    for i in range(len(y)):
        if y[i] == 1:
            if len(word) != 0:
                terms.append(word.replace(" ##", ""))
            word = x[i]
        if y[i] == 2:
            word += (" " + x[i])
    
    if len(word) != 0:
        terms.append(word.replace(" ##", ""))
            
    print("Tokens:", x)
    print("ATE:", terms)
    print(type(terms))
    
    if len(terms) != 0:
        for aspect in terms:
            aspect_text = " ".join(aspect)
            print(aspect_text)
            prediction = emotion_classifier(text)
            
            print("Aspect:", aspect, "Predicted Emotions:")
            print(prediction)
            # for emotion in prediction[0]:
            #     print(f"  - {emotion['label']}: {emotion['score']}")
            print()


d:\FYP_A+\Project_Update\ATEABSA\.venv\lib\site-packages\transformers\pipelines\text_classification.py:107: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  UserWarning,


In [12]:
import torch
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline

# Load Aspect Term Extraction (ATE) model
def predict_model_ATE(sentence, tokenizer):
    # ... (your existing ATE prediction code)
    # Replace this with your implementation for ATE prediction
    word_pieces = []
    tokens = tokenizer.tokenize(sentence)
    word_pieces += tokens

    ids = tokenizer.convert_tokens_to_ids(word_pieces)
    input_tensor = torch.tensor([ids]).to(DEVICE)

    with torch.no_grad():
        outputs = model_ATE(input_tensor, None, None)
        _, predictions = torch.max(outputs, dim=2)
    predictions = predictions[0].tolist()
    
    return word_pieces, predictions, outputs

# Load emotion detection model
classifier = pipeline("text-classification",model='bhadresh-savani/bert-base-uncased-emotion', return_all_scores=True)

def ATE_emotion_prediction(text, model_ATE, tokenizer, emotion_classifier, window_size_ratio=0.2):
    terms = []
    word = ""
    
    x, y, _ = predict_model_ATE(text, tokenizer)
    
    for i in range(len(y)):
        if y[i] == 1:
            if len(word) != 0:
                terms.append(word.replace(" ##", ""))
            word = x[i]
        if y[i] == 2:
            word += (" " + x[i])
    
    if len(word) != 0:
        terms.append(word.replace(" ##", ""))
            
    print("Tokens:", x)
    print("ATE:", terms)
    
    if len(terms) != 0:
        for aspect in terms:
            aspect_text = " ".join(aspect)
            print("Aspect:", aspect, "Predicted Emotions:")
            
            # Find the position of the aspect term in the tokenized sentence
            aspect_position = find_aspect_position(aspect.split(), x)
            
            if aspect_position != -1:
                # Calculate the dynamic window size as a fraction of the sentence length
                sentence_length = len(x)
                dynamic_window_size = max(1, int(sentence_length * window_size_ratio))
                
                # Extract the context around the aspect term
                start_context = max(0, aspect_position - dynamic_window_size)
                end_context = min(len(x), aspect_position + dynamic_window_size + len(aspect))
                context_tokens = x[start_context:end_context]
                print(start_context)
                print(end_context)
                print(context_tokens)
                # Convert context tokens back to text
                context_text = tokenizer.convert_tokens_to_string(context_tokens)
                
                # Predict emotions based on the context
                prediction = emotion_classifier(context_text)
                
                # Print the raw prediction
                print(prediction)
            
            print()

def find_aspect_position(aspect, tokens):
    aspect_len = len(aspect)
    for i in range(len(tokens) - aspect_len + 1):
        if tokens[i:i + aspect_len] == aspect:
            return i
    return -1


In [91]:
# ...

# Function to combine ATE and emotion prediction without ABSA tokenizer
def ATE_emotion_prediction(text, model_ATE, tokenizer, emotion_classifier):
    terms = []
    word = ""
    
    x, y, _ = predict_model_ATE(text, tokenizer)
    
    # Initialize variables to store sentences corresponding to each aspect
    aspect_sentences = {}
    current_aspect = []  # Initialize current_aspect here

    for i in range(len(y)):
        if y[i] == 1:
            if len(word) != 0:
                terms.append(word.replace(" ##", ""))
            word = x[i]
        elif y[i] == 2:
            word += (" " + x[i])
        elif current_aspect:  # Check if current_aspect is not empty
            aspect_text = " ".join(current_aspect)
            
            # Store the sentence corresponding to the current aspect
            if aspect_text not in aspect_sentences:
                aspect_sentences[aspect_text] = []
            aspect_sentences[aspect_text].append(word)  # Append word instead of the full text
            
            current_aspect = []

    if len(word) != 0:
        terms.append(word.replace(" ##", ""))
            
    print("Tokens:", x)
    print("ATE:", terms)
    
    if len(terms) != 0:
        for aspect in terms:
            aspect_text = " ".join(aspect)
            print("Aspect:", aspect_text)
            
            # Emotion prediction for each sentence corresponding to the aspect
            sentences_for_aspect = aspect_sentences.get(aspect_text, [])
            for sentence in sentences_for_aspect:
                print("Sentence for Emotion Prediction:", sentence)
                
                # Emotion prediction for the sentence
                emotion_prediction = emotion_classifier(sentence)
                print("Predicted Emotions for the sentence:")
                print(emotion_prediction)
                print()

# Example usage
text = "The food is delicious but the parking is hard to find"
ATE_emotion_prediction(text, model_ATE, tokenizer, classifier_emotion)


Tokens: ['the', 'food', 'is', 'delicious', 'but', 'the', 'parking', 'is', 'hard', 'to', 'find']
ATE: ['food', 'parking']
Aspect: f o o d
Aspect: p a r k i n g


In [9]:
# Example usage
text = "The food is delicious.The parking is hard to find"
ATE_emotion_prediction(text, model_ATE, tokenizer, classifier)

Tokens: ['the', 'food', 'is', 'delicious', '.', 'the', 'parking', 'is', 'hard', 'to', 'find']
ATE: ['food', 'parking']
<class 'list'>
f o o d
Aspect: food Predicted Emotions:
[[{'label': 'sadness', 'score': 0.0002427650906611234}, {'label': 'joy', 'score': 0.9982357025146484}, {'label': 'love', 'score': 0.0009890911169350147}, {'label': 'anger', 'score': 0.0001619469840079546}, {'label': 'fear', 'score': 0.00012241268996149302}, {'label': 'surprise', 'score': 0.0002479747054167092}]]

p a r k i n g
Aspect: parking Predicted Emotions:
[[{'label': 'sadness', 'score': 0.0002427650906611234}, {'label': 'joy', 'score': 0.9982357025146484}, {'label': 'love', 'score': 0.0009890911169350147}, {'label': 'anger', 'score': 0.0001619469840079546}, {'label': 'fear', 'score': 0.00012241268996149302}, {'label': 'surprise', 'score': 0.0002479747054167092}]]



In [20]:
import nltk
from nltk import word_tokenize, sent_tokenize, pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


def extract_aspect_sentences(text, aspects):
    sentences = sent_tokenize(text)

    aspect_sentences = {aspect: [] for aspect in aspects}

    for sentence in sentences:
        words = word_tokenize(sentence)
        pos_tags = pos_tag(words)

        for aspect in aspects:
            aspect_keywords = [word.lower() for word in aspect.split()]

            if any(keyword in [word.lower() for word, pos in pos_tags] for keyword in aspect_keywords):
                aspect_sentences[aspect].append(sentence)

    return aspect_sentences

# Example usage:
text = "I love the food. The staff is rude. The parking is hard to find"
aspects_to_extract = ["food", "staff","parking"]

result = extract_aspect_sentences(text, aspects_to_extract)
print(result)




{'food': ['I love the food.'], 'staff': ['The staff is rude.'], 'parking': ['The parking is hard to find']}
food sentences:
- I love the food.

staff sentences:
- The staff is rude.

parking sentences:
- The parking is hard to find



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [66]:
import torch
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline
import nltk
from nltk import word_tokenize, sent_tokenize, pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def extract_aspect_sentences(text, aspects):
    sentences = sent_tokenize(text)

    aspect_sentences = {aspect: [] for aspect in aspects}

    for sentence in sentences:
        words = word_tokenize(sentence)
        pos_tags = pos_tag(words)

        for aspect in aspects:
            aspect_keywords = [word.lower() for word in aspect.split()]

            if any(keyword in [word.lower() for word, pos in pos_tags] for keyword in aspect_keywords):
                aspect_sentences[aspect].append(sentence)

    return aspect_sentences


# Load Aspect Term Extraction (ATE) model
def predict_model_ATE(sentence, tokenizer):
    # ... (your existing ATE prediction code)
    # Replace this with your implementation for ATE prediction
    word_pieces = []
    tokens = tokenizer.tokenize(sentence)
    word_pieces += tokens

    ids = tokenizer.convert_tokens_to_ids(word_pieces)
    input_tensor = torch.tensor([ids]).to(DEVICE)

    with torch.no_grad():
        outputs = model_ATE(input_tensor, None, None)
        _, predictions = torch.max(outputs, dim=2)
    predictions = predictions[0].tolist()
    
    return word_pieces, predictions, outputs

# Load emotion detection model
classifier = pipeline("text-classification",model='bhadresh-savani/bert-base-uncased-emotion', return_all_scores=True)

# Function to combine ATE and emotion prediction
def ATE_emotion_prediction(text, model_ATE, tokenizer, emotion_classifier):

    terms = []
    word = ""

    # Assuming that predict_model_ATE returns the tokens x, labels y, and other information (_)
    x, y, _ = predict_model_ATE(text, tokenizer)

    for i in range(len(y)):
        if y[i] == 1:
            # If the label is 1, it indicates the start of a new aspect term
            if len(word) != 0:
                # If there's an existing word, append it to the terms list after removing " ##"
                terms.append(word.replace(" ##", ""))
            # Start a new word with the current token
            word = x[i]
        if y[i] == 2:
            word += (" " + x[i])

    if len(word) != 0:
        # print(terms)
        terms.append(word.replace(" ##", ""))

    combined_terms = []
    for term in terms:
        if "#" in term:
            if combined_terms:
                combined_terms[-1] += term
        else:
            combined_terms.append(term)

    # Remove hashtags from the combined terms
    terms_without_hashtag = [term.replace("#", "") for term in combined_terms]
    # Replace specific words with desired format
    for i in range(len(terms_without_hashtag)):
        if  terms_without_hashtag[i] == 'nasilemak':
            terms_without_hashtag[i] = 'nasi lemak'
           


    # Print the original tokens and the extracted aspect terms
    print("Tokens:", x)
    print("ATE:", terms_without_hashtag)
    print()

    
    result = extract_aspect_sentences(text, terms_without_hashtag)
    print(type(result))
    print(result)
    for aspect, sentences in result.items():
        print(f"{aspect} sentences:")
        
        # Loop over each sentence for the current aspect
        for sentence in sentences:
            print(f"- {sentence}")
            
            # Assuming you have an emotion_classifier function
            emotion_result = emotion_classifier(sentence)
            print(type(emotion_result))
            print(f"Emotion result for '{aspect}': {emotion_result}")
    
        print()  # Print a new line after all sentences for the current aspect



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
d:\FYP_A+\Project_Update\ATEABSA\.venv\lib\site-packages\transformers\pipelines\text_classification.py:107: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  UserWarning,


In [67]:
# Example usage
text = "The queue is very long but worth it. Don't talk about car parking; it's crowded. It's better to take the LRT, stop near Kampung Bharu, and then walk around 3 minutes; the distance is short. Their system is very efficient; you pay first before eating."

ATE_emotion_prediction(text, model_ATE, tokenizer, classifier)

Tokens: ['the', 'queue', 'is', 'very', 'long', 'but', 'worth', 'it', '.', 'don', "'", 't', 'talk', 'about', 'car', 'parking', ';', 'it', "'", 's', 'crowded', '.', 'it', "'", 's', 'better', 'to', 'take', 'the', 'l', '##rt', ',', 'stop', 'near', 'kam', '##pu', '##ng', 'b', '##har', '##u', ',', 'and', 'then', 'walk', 'around', '3', 'minutes', ';', 'the', 'distance', 'is', 'short', '.', 'their', 'system', 'is', 'very', 'efficient', ';', 'you', 'pay', 'first', 'before', 'eating', '.']
ATE: ['queue', 'parking', 'system']

<class 'dict'>
{'queue': ['The queue is very long but worth it.'], 'parking': ["Don't talk about car parking; it's crowded."], 'system': ['Their system is very efficient; you pay first before eating.']}
queue sentences:
- The queue is very long but worth it.
<class 'list'>
Emotion result for 'queue': [[{'label': 'sadness', 'score': 0.0016604585107415915}, {'label': 'joy', 'score': 0.992912232875824}, {'label': 'love', 'score': 0.0012597484746947885}, {'label': 'anger', 'sco

In [61]:
import csv
import pandas as pd

# Function to read CSV file and process each row
def process_csv_file(input_file_path, output_file_path):
    # Create an empty DataFrame to store the results
    results_df = pd.DataFrame(columns=['Text', 'Aspect', 'Emotion'])

    with open(input_file_path, 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            text = row['Text']
            
            # Perform ATE and emotion prediction for each row
            terms, result = ATE_emotion_prediction(text, model_ATE, tokenizer, classifier)

            # Append the results to the DataFrame
            results_df = results_df.append({'Text': text, 'Aspect': terms, 'Emotion': result}, ignore_index=True)

    # Write the results to an Excel file
    results_df.to_excel(output_file_path, index=False)

# Example usage:
csv_file_path = 'CSV_Review.csv' 
output_excel_path = 'CSV_ReviewDashboard.csv'  
process_csv_file(csv_file_path, output_excel_path)


Tokens: ['when', 'it', 'arrives', 'around', 'before', 'dusk', ',', 'the', 'line', 'is', 'not', 'long', '.', 'but', 'here', 'is', 'a', 'prayer', 'room', ',', 'don', "'", 't', 'worry', '.']
ATE: ['line', 'prayer room']

line sentences:
- When it arrives around before dusk, the line is not long.
Emotion result for 'line': [[{'label': 'sadness', 'score': 0.008317738771438599}, {'label': 'joy', 'score': 0.01027082558721304}, {'label': 'love', 'score': 0.0015612308634445071}, {'label': 'anger', 'score': 0.04784228280186653}, {'label': 'fear', 'score': 0.9286301732063293}, {'label': 'surprise', 'score': 0.0033777335193008184}]]

prayer room sentences:
- But here is a prayer room, don't worry.
Emotion result for 'prayer room': [[{'label': 'sadness', 'score': 0.009689624421298504}, {'label': 'joy', 'score': 0.8718135356903076}, {'label': 'love', 'score': 0.0990135595202446}, {'label': 'anger', 'score': 0.01375632919371128}, {'label': 'fear', 'score': 0.0045320275239646435}, {'label': 'surprise'

TypeError: cannot unpack non-iterable NoneType object

In [9]:
import torch
import copy
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline
import nltk
from nltk import word_tokenize, sent_tokenize, pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
classifier = pipeline("text-classification",model='bhadresh-savani/bert-base-uncased-emotion', return_all_scores=True)

def extract_aspect_sentences(text, aspects):
    sentences = sent_tokenize(text)
    aspect_sentences = {aspect: [] for aspect in aspects}

    for sentence in sentences:
        words = word_tokenize(sentence)
        pos_tags = pos_tag(words)

        for aspect in aspects:
            aspect_keywords = [word.lower() for word in aspect.split()]

            if any(keyword in [word.lower() for word, pos in pos_tags] for keyword in aspect_keywords):
                aspect_sentences[aspect].append(sentence)

    return aspect_sentences


def predict_model_ATE(sentence, tokenizer):
    word_pieces = []
    tokens = tokenizer.tokenize(sentence)
    word_pieces += tokens

    ids = tokenizer.convert_tokens_to_ids(word_pieces)
    input_tensor = torch.tensor([ids]).to(DEVICE)

    with torch.no_grad():
        outputs = model_ATE(input_tensor, None, None)
        _, predictions = torch.max(outputs, dim=2)
    predictions = predictions[0].tolist()
    
    return word_pieces, predictions, outputs


def ATE_emotion_prediction(text, model_ATE, tokenizer, emotion_classifier):

    terms = []
    word = ""

    # Assuming that predict_model_ATE returns the tokens x, labels y, and other information (_)
    x, y, _ = predict_model_ATE(text, tokenizer)

    for i in range(len(y)):
        if y[i] == 1:
            # If the label is 1, it indicates the start of a new aspect term
            if len(word) != 0:
                # If there's an existing word, append it to the terms list after removing " ##"
                terms.append(word.replace(" ##", ""))
            # Start a new word with the current token
            word = x[i]
        if y[i] == 2:
            word += (" " + x[i])

    if len(word) != 0:
        # print(terms)
        terms.append(word.replace(" ##", ""))

    combined_terms = []
    for term in terms:
        if "#" in term:
            if combined_terms:
                combined_terms[-1] += term
        else:
            combined_terms.append(term)

    # Remove hashtags from the combined terms
    terms_without_hashtag = [term.replace("#", "") for term in combined_terms]
    # Replace specific words with desired format
    for i in range(len(terms_without_hashtag)):
        if  terms_without_hashtag[i] == 'nasilemak':
            terms_without_hashtag[i] = 'nasi lemak'
           


    # Print the original tokens and the extracted aspect terms
    # print("Tokens:", x)
    print("ATE:", terms_without_hashtag)
    print()

    
    result = extract_aspect_sentences(text, terms_without_hashtag)
    emotion_dict = {}
    emotion_dict = copy.deepcopy(result)
    
    # print(type(result))
    print(f"Result : {result}")
    for aspect, sentences in result.items():
        # print(f"{aspect} sentences:")
        # print("iteration aspect sentence")
        # Loop over each sentence for the current aspect
        for sentence in sentences:
            # print(f"- {sentence}")
            
            # Assuming you have an emotion_classifier function
            emotion_result = emotion_classifier(sentence)
            emotion_dict[aspect]+= emotion_result
            

            # print(emotion_result)
            # print(print(emotion_dict))
     
        print()  # Print a new line after all sentences for the current aspect
    
    return terms_without_hashtag, emotion_dict


def process_csv_file(input_file_path, output_file_path):

    results_df = pd.DataFrame(columns=['Aspect', 'Emotion'])
    with open(input_file_path, 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            text = row['Text']
            
            # Perform ATE and emotion prediction for each row
            terms, result = ATE_emotion_prediction(text, model_ATE, tokenizer, classifier)

            # Append the results to the DataFrame
            results_df = results_df.append({'Text': text, 'Aspect': terms, 'Emotion': result}, ignore_index=True)

    # Write the results to an Excel file
    results_df.to_excel(output_file_path, index=False)




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
d:\FYP_A+\Project_Update\ATEABSA\.venv\lib\site-packages\transformers\pipelines\text_classification.py:107: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  UserWarning,
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [8]:
import pandas as pd
import csv
results_df = pd.DataFrame(columns=['Aspect', 'Emotion'])
with open('CSV_Review.csv', 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        text = row['Text']
        print(text)
        # Perform ATE and emotion prediction for each row
        terms, result = ATE_emotion_prediction(text, model_ATE, tokenizer, classifier)
        # print (terms)
        print(f"Hola :{result}")
        # Append the results to the DataFrame
        results_df = results_df.append({'Text': text, 'Aspect': terms, 'Emotion': result}, ignore_index=True)
        # results_df.to_csv('dashboard.csv', index=False)
       

When it arrives around before dusk, the line is not long. But here is a prayer room, don't worry.
ATE: ['line', 'prayer room']

Result : {'line': ['When it arrives around before dusk, the line is not long.'], 'prayer room': ["But here is a prayer room, don't worry."]}


Hola :{'line': ['When it arrives around before dusk, the line is not long.', [{'label': 'sadness', 'score': 0.008317738771438599}, {'label': 'joy', 'score': 0.01027082558721304}, {'label': 'love', 'score': 0.0015612308634445071}, {'label': 'anger', 'score': 0.04784228280186653}, {'label': 'fear', 'score': 0.9286301732063293}, {'label': 'surprise', 'score': 0.0033777335193008184}]], 'prayer room': ["But here is a prayer room, don't worry.", [{'label': 'sadness', 'score': 0.009689624421298504}, {'label': 'joy', 'score': 0.8718135356903076}, {'label': 'love', 'score': 0.0990135595202446}, {'label': 'anger', 'score': 0.01375632919371128}, {'label': 'fear', 'score': 0.0045320275239646435}, {'label': 'surprise', 'score': 0.00

KeyboardInterrupt: 

In [14]:
# Create an empty DataFrame to store the results
final_df = pd.DataFrame()

# Iterate through each aspect in the result dictionary
for aspect, data in result.items():
    # Extract the emotions list from the aspect entry
    emotions_list = data[1]

    # Custom sorting function based on the 'score' values
    def custom_sort(emotion):
        return float(emotion['score'])

    # Sort the emotions list based on the 'score' values in descending order using the custom sorting function
    sorted_emotion_data = sorted(emotions_list, key=custom_sort, reverse=True)

    # Round the 'score' values to 3 decimal places in the sorted list
    rounded_sorted_emotion_data = [{'label': entry['label'], 'score': round(float(entry['score']), 3)} for entry in sorted_emotion_data]

    # Create a DataFrame for the current aspect
    df = pd.DataFrame(rounded_sorted_emotion_data)

    # Add the aspect name as a new column
    df['Aspect'] = aspect

    # Reshape the DataFrame for the desired output
    df_output = df.set_index(['Aspect', df.groupby('Aspect').cumcount() + 1]).unstack().sort_index(axis=1, level=1)

    # Flatten MultiIndex columns
    df_output.columns = [f"{col[0]}_{col[1]}" for col in df_output.columns]

    # Reset index for better formatting
    df_output.reset_index(inplace=True)

    # Append the current aspect's DataFrame to the final DataFrame
    final_df = final_df.append(df_output, ignore_index=True)

# Print the final DataFrame
print(final_df)

    Aspect  label_1  score_1 label_2  score_2 label_3  score_3   label_4  \
0   sotong     love    0.811     joy    0.183   anger    0.003  surprise   
1   sambal     love    0.811     joy    0.183   anger    0.003  surprise   
2     rice  sadness    0.763   anger    0.228    fear    0.004       joy   
3  parking    anger    0.397     joy    0.315    fear    0.171   sadness   
4    price      joy    0.996    love    0.001   anger    0.001   sadness   

   score_4   label_5  score_5   label_6  score_6  
0    0.001   sadness    0.001      fear    0.001  
1    0.001   sadness    0.001      fear    0.001  
2    0.003  surprise    0.001      love    0.001  
3    0.092  surprise    0.015      love    0.009  
4    0.001      fear    0.000  surprise    0.000  


In [33]:
#CSV FILE
import csv
import torch
import copy
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline
import nltk
from nltk import word_tokenize, sent_tokenize, pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
classifier = pipeline("text-classification",model='bhadresh-savani/bert-base-uncased-emotion', return_all_scores=True)

def extract_aspect_sentences(text, aspects):
    sentences = sent_tokenize(text)
    aspect_sentences = {aspect: [] for aspect in aspects}

    for sentence in sentences:
        words = word_tokenize(sentence)
        pos_tags = pos_tag(words)
        for aspect in aspects:
            aspect_keywords = [word.lower() for word in aspect.split()]
            if any(keyword in [word.lower() for word, pos in pos_tags] for keyword in aspect_keywords):
                aspect_sentences[aspect].append(sentence)

    return aspect_sentences


def predict_model_ATE(sentence, tokenizer):
    word_pieces = []
    tokens = tokenizer.tokenize(sentence)
    word_pieces += tokens
    ids = tokenizer.convert_tokens_to_ids(word_pieces)
    input_tensor = torch.tensor([ids]).to(DEVICE)
    with torch.no_grad():
        outputs = model_ATE(input_tensor, None, None)
        _, predictions = torch.max(outputs, dim=2)
    predictions = predictions[0].tolist()
    
    return word_pieces, predictions, outputs


def ATE_emotion_prediction_multi(text, model_ATE, tokenizer, emotion_classifier):
    terms = []
    word = ""
    x, y, _ = predict_model_ATE(text, tokenizer)
    
    for i in range(len(y)):
        if y[i] == 1:
            # If the label is 1, it indicates the start of a new aspect term
            if len(word) != 0:
                # If there's an existing word, append it to the terms list after removing " ##"
                terms.append(word.replace(" ##", ""))
            # Start a new word with the current token
            word = x[i]
        if y[i] == 2:
            word += (" " + x[i])

    if len(word) != 0:
        terms.append(word.replace(" ##", ""))

    combined_terms = []
    for term in terms:
        if "#" in term:
            if combined_terms:
                combined_terms[-1] += term
        else:
            combined_terms.append(term)

    terms_without_hashtag = [term.replace("#", "") for term in combined_terms]
    for i in range(len(terms_without_hashtag)):
        if  terms_without_hashtag[i] == 'nasilemak':
            terms_without_hashtag[i] = 'nasi lemak'
           
    # Print the original tokens and the extracted aspect terms
    # print("Tokens:", x)
    # print("ATE:", terms_without_hashtag)
    # print()

    result = extract_aspect_sentences(text, terms_without_hashtag)
    emotion_dict = {}
    emotion_dict = copy.deepcopy(result)
    
    # print(type(result))
    # print(f"Result : {result}")
    for aspect, sentences in result.items():
        # print(f"{aspect} sentences:")
        # print("iteration aspect sentence")
        # Loop over each sentence for the current aspect
        for sentence in sentences:
            # print(f"- {sentence}")

            emotion_result = emotion_classifier(sentence)
            emotion_dict[aspect]+= emotion_result

        print()  

    return terms_without_hashtag, emotion_dict


def process_csv_file(input_file_path, output_file_path):
    results_df = pd.DataFrame(columns=['Aspect', 'Label1', 'Score1', 'Label2', 'Score2', 'Label3', 'Score3', 'Label4', 'Score4', 'Label5', 'Score5', 'Label6', 'Score6'])
    
    with open(input_file_path, 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            text = row['Text']
            terms, result_prediction = ATE_emotion_prediction(text, model_ATE, tokenizer, classifier)
            aspect_df = pd.DataFrame()
            for aspect, data in result_prediction.items():
                emotions_list = data[1]

                # Custom sorting function based on the 'score' values
                def custom_sort(emotion):
                    try:
                        return float(emotion['score'])
                    except (ValueError, TypeError):
                        return 0.0

                sorted_emotion_data = sorted(emotions_list, key=custom_sort, reverse=True)
                rounded_sorted_emotion_data = [{'label': entry['label'], 'score': round(float(entry['score']), 3)} for entry in sorted_emotion_data]

                # Create a new row for each aspect
                new_row = {'Aspect': aspect}
                for i, entry in enumerate(rounded_sorted_emotion_data):
                    new_row[f'Label{i+1}'] = entry['label']
                    new_row[f'Score{i+1}'] = entry['score']

                # Append the new row to the DataFrame
                results_df = results_df.append(new_row, ignore_index=True)

    print(results_df.head())       
    results_df.to_csv(output_file_path, index=False, encoding='utf-8')

# Example usage:
input_file_path = 'CSV_Review.csv'
output_file_path = 'LALALa.csv'
process_csv_file(input_file_path, output_file_path)
           



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
d:\FYP_A+\Project_Update\ATEABSA\.venv\lib\site-packages\transformers\pipelines\text_classification.py:107: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  UserWarning,



















































































































































































































































































































































































         Aspect Label1  Score1   Label2  Score2    Label3  Score3   Label4  \
0          line   fear   0.929    anger   0.048       joy   0.010  sadness   
1   prayer room    joy   0.872     love   0.099     anger   0.014  sadness   
2   dining area    joy   0.995     love   0.002  surprise   0.001     fear   
3          food    joy   0.990  sadness   0.004     anger   0.002     fear   
4  filter water    joy   0.985    anger   0.007   sadness   0.003     love   

   Score4    Label5  Score5    Label6  Score6  
0   0.008  surprise   0.003      love   0.002  
1   0.010      fear   0.005  surprise   0.001  
2   0.001   sadnes

In [ ]:
def process_csv_file(input_file_path, output_file_path):
    results_df = pd.DataFrame(columns=['Aspect', 'Label1','Score1','Label2','Score2','Label3','Score3','Label4','Score4','Label5','Score5','Label6','Score6'])
    
    with open(input_file_path, 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            text = row['Text']
            terms, result_prediction = ATE_emotion_prediction(text, model_ATE, tokenizer, classifier)
            print("Result lepas predict: ",result_prediction)
            aspect_df = pd.DataFrame()
            for aspect, data in result_prediction.items():
                print("ASPECT : ",aspect)
                emotions_list = data[1]

                # Custom sorting function based on the 'score' values
                # def custom_sort(emotion):
                #     return float(emotion['score'])

                def custom_sort(emotion):
                    try:
                        return float(emotion['score'])
                    except (ValueError, TypeError):
                        return 0.0 

                print("nak amik list emotion ja: ",emotions_list)
                sorted_emotion_data = sorted(emotions_list, key=custom_sort, reverse=True)
                rounded_sorted_emotion_data = [{'label': entry['label'], 'score': round(float(entry['score']), 3)} for entry in sorted_emotion_data]
                print("Round value score : ", rounded_sorted_emotion_data)

                # Add a new row
                new_row = {'Aspect': aspect, 'Label1': , 'Score1': , 'Label2': , 'Score2': , 'Label3': , 'Score3': , 'Label4': , 'Score4': , 'Label5': , 'Score5': , 'Label6': , 'Score6':  }
                results_df = df.append(new_row, ignore_index=True)

                df = pd.DataFrame(rounded_sorted_emotion_data)
                results['Aspect'] = aspect
                df_output = df.set_index(['Aspect', df.groupby('Aspect').cumcount() + 1]).unstack().sort_index(axis=1, level=1)
                df_output.columns = [f"{col[0]}_{col[1]}" for col in df_output.columns]
                df_output.reset_index(inplace=True)
                aspect_df = aspect_df.append(df_output, ignore_index=True)

            results_df = results_df.append({'Text': text, 'Aspect': terms, 'Emotion': aspect_df}, ignore_index=True)
            
    results_df.to_csv(output_file_path, index=False, encoding='utf-8')

In [22]:
def sorted_round(list_emotion):
    def custom_sort(emotion):
            try:
                return float(emotion['score'])
            except (ValueError, TypeError):
                return 0.0

    sorted_emotion_data = sorted(list_emotion, key=custom_sort, reverse=True)
    rounded_sorted_emotion_data = [{'label': entry['label'], 'score': round(float(entry['score']),3)} for entry in sorted_emotion_data]

    return rounded_sorted_emotion_data
text = "I love the food. Parking is hard"

prediction_result = ATE_emotion_prediction(text, model_ATE, tokenizer, classifier)
print(type(prediction_result))
print(prediction_result[1])
results_df = pd.DataFrame(columns=['Aspect', 'Label1', 'Score1', 'Label2', 'Score2', 'Label3', 'Score3', 'Label4', 'Score4', 'Label5', 'Score5', 'Label6', 'Score6'])
            
for aspect, aspect_emotions in prediction_result[1].items():
    print(f"\nAspect: {aspect}")
    if aspect_emotions:
        emotion_list = aspect_emotions[1]  
        rounded_sorted_emotion_data = sorted_round(emotion_list)

        new_row = {'Aspect': aspect}
        for i, entry in enumerate(rounded_sorted_emotion_data):
            new_row[f'Label{i+1}'] = entry['label']
            new_row[f'Score{i+1}'] = entry['score']
        results_df = results_df.append(new_row, ignore_index=True)

print("Emotion Prediction DataFrame:")
print(results_df)




ATE: ['food', 'parking']

Result : {'food': ['I love the food.'], 'parking': ['Parking is hard']}


<class 'tuple'>
{'food': ['I love the food.', [{'label': 'sadness', 'score': 0.001409263932146132}, {'label': 'joy', 'score': 0.9703583121299744}, {'label': 'love', 'score': 0.024186555296182632}, {'label': 'anger', 'score': 0.0032605831511318684}, {'label': 'fear', 'score': 0.0002018398226937279}, {'label': 'surprise', 'score': 0.0005834508338011801}]], 'parking': ['Parking is hard', [{'label': 'sadness', 'score': 0.008521218784153461}, {'label': 'joy', 'score': 0.0030597879085689783}, {'label': 'love', 'score': 0.0005783014930784702}, {'label': 'anger', 'score': 0.9835397005081177}, {'label': 'fear', 'score': 0.0037728159222751856}, {'label': 'surprise', 'score': 0.0005281605990603566}]]}

Aspect: food

Aspect: parking
Emotion Prediction DataFrame:
    Aspect Label1  Score1   Label2  Score2 Label3  Score3   Label4  Score4  \
0     food    joy   0.970     love   0.024  anger   0.003  sa